In [ ]:
 import torch
from transformers import pipeline

device = 0 if torch.cuda.is_available() else -1

pipe = pipeline("text-classification", model="finiteautomata/bertweet-base-sentiment-analysis", device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Device set to use cuda:0


In [ ]:
import pandas as pd
train = pd.read_csv("/content/sentiment_train.csv")
test = pd.read_csv("/content/sentiment_test.csv")
test

,title,sentiment
0,What is melatonin -- and should you take it to...,Neutral
1,The world in 2200,Neutral
2,The real story behind Archimedes' Eureka!,Neutral
3,Are you technically fit to parent?,Neutral
4,Big data is better data,Positive
...,...,...
1261,"What ""Orwellian"" really means",Neutral
1262,"1,000 TED Talks in six words",Neutral
1263,We actually have a shot at stopping the climat...,Positive
1264,The billion-dollar campaign to electrify trans...,Positive


In [ ]:
data = ["I love you", "I hate you"]
pipe(data)

[{'label': 'POS', 'score': 0.9916695356369019},
 {'label': 'NEG', 'score': 0.9806600213050842}]

In [ ]:
from tqdm.auto import tqdm

In [ ]:
# Wrap tqdm around the pandas apply function
tqdm.pandas()

test['predicted_sentiment'] = test['title'].progress_apply(lambda text: pipe(text)[0]['label'])

  0%|          | 0/1266 [00:00<?, ?it/s]

In [ ]:
test

,title,sentiment,predicted_sentiment
0,What is melatonin -- and should you take it to...,Neutral,NEU
1,The world in 2200,Neutral,NEU
2,The real story behind Archimedes' Eureka!,Neutral,NEU
3,Are you technically fit to parent?,Neutral,NEU
4,Big data is better data,Positive,POS
...,...,...,...
1261,"What ""Orwellian"" really means",Neutral,NEU
1262,"1,000 TED Talks in six words",Neutral,NEU
1263,We actually have a shot at stopping the climat...,Positive,POS
1264,The billion-dollar campaign to electrify trans...,Positive,NEU


In [ ]:
label_mapping1 = {'Positive': 0, 'Negative': 1, 'Neutral': 2}
label_mapping2 = {'POS':0, 'NEG':1, 'NEU':2}
test['sentiment_numeric'] = test['sentiment'].map(label_mapping1).fillna(-1).astype(int)
test['predicted_sentiment_numeric'] = test['predicted_sentiment'].map(label_mapping2).fillna(-1).astype(int)


In [ ]:
test

,title,sentiment,sentiment_numeric,predicted_sentiment,predicted_sentiment_numeric
0,What is melatonin -- and should you take it to...,Neutral,2,NEU,2
1,The world in 2200,Neutral,2,NEU,2
2,The real story behind Archimedes' Eureka!,Neutral,2,NEU,2
3,Are you technically fit to parent?,Neutral,2,NEU,2
4,Big data is better data,Positive,0,POS,0
...,...,...,...,...,...
1261,"What ""Orwellian"" really means",Neutral,2,NEU,2
1262,"1,000 TED Talks in six words",Neutral,2,NEU,2
1263,We actually have a shot at stopping the climat...,Positive,0,POS,0
1264,The billion-dollar campaign to electrify trans...,Positive,0,NEU,2


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, f1_score
accuracy = accuracy_score(test['sentiment_numeric'], test['predicted_sentiment_numeric'])
recall = recall_score(test['sentiment_numeric'], test['predicted_sentiment_numeric'], average='weighted')
# auc = roc_auc_score(test['sentiment_numeric'], test['predicted_sentiment_numeric'], multi_class='ovr')
f1 = f1_score(test['sentiment_numeric'], test['predicted_sentiment_numeric'], average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
# print(f"AUC: {auc}")
print(f"F1 Score: {f1}")

Accuracy: 0.5268562401263823
Recall: 0.5268562401263823
F1 Score: 0.51474503762957


# FINE_TUNING


In [ ]:
!pip install --upgrade datasets transformers evaluate sentencepiece accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

file_dict = {
  "train" : "/content/sentiment_test.csv",
  "test" : "/content/sentiment_test.csv"
}

dataset = load_dataset(
  'csv',
  data_files=file_dict,
  delimiter=',',
  column_names=['title', 'sentiment'],
  skiprows=1,
)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'sentiment'],
        num_rows: 1266
    })
    test: Dataset({
        features: ['title', 'sentiment'],
        num_rows: 1266
    })
})

In [ ]:
label_mapping1 = {'Neutral': 0, 'Negative': 1, 'Positive': 2}
def map_sentiment(example):
  example['sentiment'] = label_mapping1.get(example['sentiment'], example['sentiment'])
  return example

dataset = dataset.map(map_sentiment)

Map:   0%|          | 0/1266 [00:00<?, ? examples/s]

Map:   0%|          | 0/1266 [00:00<?, ? examples/s]

In [ ]:
set_seed = 42

In [ ]:
from transformers import AutoTokenizer

model_path = 'finiteautomata/bertweet-base-sentiment-analysis'

tokenizer = AutoTokenizer.from_pretrained(model_path)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [ ]:
def preprocess_function(examples):
   return tokenizer(examples["title"], truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/1266 [00:00<?, ? examples/s]

Map:   0%|          | 0/1266 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis", num_labels=3)


In [ ]:
# import numpy as np
# import evaluate
# from evaluate import load

# load_metric = evaluate.load

# def compute_metrics(eval_pred):
#    load_accuracy = load_metric("accuracy")
#    load_f1 = load_metric("f1")

#    logits, labels = eval_pred
#    predictions = np.argmax(logits, axis=-1)
#    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
#    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
#    return {"accuracy": accuracy, "f1": f1}


In [ ]:
# import numpy as np
# import evaluate
# from evaluate import load

# load_metric = evaluate.load

# def compute_metrics(eval_pred):
#     load_accuracy = load_metric("accuracy")
#     load_f1 = load_metric("f1")

#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
#     # Check if predictions is a single value, if so, convert it to a list
#     if not isinstance(predictions, (np.ndarray, list)):  # or any other iterable type if needed
#         predictions = [predictions]
#     # Change average to 'weighted' for multiclass classification
#     f1 = load_f1.compute(predictions=predictions, references=labels, average='weighted')["f1"]
#     return {"accuracy": accuracy, "f1": f1}

In [ ]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score
from transformers import EvalPrediction

def single_label_metrics(predictions, labels):
    """
    Calculates metrics for single-label classification.

    Args:
        predictions: Model predictions (logits or probabilities).
        labels: True labels.

    Returns:
        A dictionary containing the calculated metrics.
    """
    # For single-label, get the predicted class index
    predicted_labels = np.argmax(predictions, axis=1)

    # Calculate metrics
    accuracy = accuracy_score(labels, predicted_labels)
    f1 = f1_score(labels, predicted_labels, average='weighted')  # Use 'weighted' for multi-class
    recall = recall_score(labels, predicted_labels, average='weighted')  # Use 'weighted' for multi-class

    # Return as dictionary
    metrics = {
        'f1': f1,
        'recall': recall,
        'accuracy': accuracy
    }
    return metrics

def compute_metrics(p: EvalPrediction):
    """
    Wrapper function for computing metrics using the single_label_metrics function.

    Args:
        p: EvalPrediction object containing predictions and labels.

    Returns:
        A dictionary containing the calculated metrics.
    """
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = single_label_metrics(predictions=preds, labels=p.label_ids)
    return result

In [ ]:
from google.colab import userdata
WANDB_KEY = userdata.get('WANDB_KEY')

In [ ]:
import wandb
wandb.login(key=WANDB_KEY)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: czarnybaranie1 (czarnybaranie1-sgh-warsaw-school-of-economics). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Rename columns to text and labels (needed for model setup)
tokenized_dataset = tokenized_dataset.rename_column("title", "text")
tokenized_dataset = tokenized_dataset.rename_column("sentiment", "labels")

In [ ]:
import os
# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="sentiment-classification"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="checkpoint"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

In [ ]:
from transformers import TrainingArguments, Trainer

repo_name = "finetuning-sentiment-model-tedx"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
   evaluation_strategy="steps",
   eval_steps=10,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset['train'],
   eval_dataset=tokenized_dataset['test'],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-152-fc8e46f14cd5>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
tokenized_dataset['test']['labels']

['NEU',
 'NEU',
 'NEU',
 'NEU',
 'POS',
 'POS',
 'POS',
 'NEU',
 'NEU',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'NEU',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'NEU',
 'POS',
 'NEU',
 'POS',
 'NEU',
 'POS',
 'POS',
 'POS',
 'POS',
 'NEG',
 'NEU',
 'POS',
 'NEU',
 'POS',
 'POS',
 'POS',
 'POS',
 'NEU',
 'NEU',
 'NEU',
 'NEU',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'NEU',
 'NEU',
 'NEU',
 'POS',
 'NEG',
 'NEU',
 'NEU',
 'POS',
 'POS',
 'NEG',
 'NEU',
 'NEG',
 'NEU',
 'NEU',
 'POS',
 'NEU',
 'POS',
 'POS',
 'POS',
 'POS',
 'NEU',
 'NEU',
 'NEU',
 'POS',
 'POS',
 'NEU',
 'POS',
 'POS',
 'NEU',
 'NEU',
 'NEU',
 'POS',
 'POS',
 'NEU',
 'NEG',
 'NEU',
 'NEU',
 'POS',
 'POS',
 'NEU',
 'POS',
 'POS',
 'NEG',
 'POS',
 'NEG',
 'NEU',
 'POS',
 'POS',
 'POS',
 'NEU',
 'NEU',
 'NEU',
 'NEU',
 'POS',
 'NEU',
 'NEU',
 'POS',
 'POS',
 'POS',
 'NEU',
 'NEU',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'NEU',
 'POS',
 'NEG',
 'NEU',
 'POS',
 'POS',
 'NEU',
 'NEU',
 'POS',
 'POS',
 'NEG',


In [ ]:
trainer.train()


Step,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
100,No log,0.057432,0.986669,None,0.986572


wandb: Adding directory to artifact (./finetuning-sentiment-model-tedx/checkpoint-80)... Done. 35.4s
Trainer is attempting to log a value of "None" of type <class 'NoneType'> for key "eval/roc_auc" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
wandb: Adding directory to artifact (./finetuning-sentiment-model-tedx/checkpoint-160)... Done. 33.3s
wandb: Adding directory to artifact (./finetuning-sentiment-model-tedx/checkpoint-160)... No files have been modified since last commit. Skipping to prevent empty commit.
Done. 17.4s


TrainOutput(global_step=160, training_loss=0.07429133057594299, metrics={'train_runtime': 210.709, 'train_samples_per_second': 12.017, 'train_steps_per_second': 0.759, 'total_flos': 21473549361684.0, 'train_loss': 0.07429133057594299, 'epoch': 2.0})

In [ ]:
trainer.train()


Step,Training Loss,Validation Loss,F1,Recall,Accuracy
10,No log,0.103875,0.973707,0.973144,0.973144
20,No log,0.102310,0.981875,0.981833,0.981833
30,No log,0.095744,0.975939,0.976303,0.976303
40,No log,0.057179,0.984240,0.984202,0.984202
50,No log,0.077202,0.983458,0.983412,0.983412
60,No log,0.036865,0.992094,0.992101,0.992101
70,No log,0.172184,0.966555,0.966825,0.966825
80,No log,0.024761,0.993676,0.993681,0.993681
90,No log,0.042172,0.992901,0.992891,0.992891
100,No log,0.052903,0.990534,0.990521,0.990521


wandb: Adding directory to artifact (./finetuning-sentiment-model-tedx/checkpoint-80)... Done. 36.3s
wandb: Adding directory to artifact (./finetuning-sentiment-model-tedx/checkpoint-160)... Done. 12.4s
wandb: Adding directory to artifact (./finetuning-sentiment-model-tedx/checkpoint-160)... No files have been modified since last commit. Skipping to prevent empty commit.
Done. 12.2s


TrainOutput(global_step=160, training_loss=0.03513135612010956, metrics={'train_runtime': 184.9482, 'train_samples_per_second': 13.69, 'train_steps_per_second': 0.865, 'total_flos': 21473549361684.0, 'train_loss': 0.03513135612010956, 'epoch': 2.0})

In [ ]:
trainer.save_model("/content/finetuning-sentiment-model-tedx/final_model")

# INFERENCE

In [ ]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained('CzarnyBaranie/bart-finetuned-for-tedx-topics')

# model = AutoModelForSequenceClassification.from_pretrained("CzarnyBaranie/bart-finetuned-for-tedx-topics")

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="CzarnyBaranie/finetuning-sentiment-model-tedx", device=0)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Device set to use cuda:0


In [ ]:
pipe(["I love this move", "This movie sucks!"])

[{'label': 'POS', 'score': 0.9999136924743652},
 {'label': 'NEG', 'score': 0.9997326731681824}]

In [ ]:
import pandas as pd
test = pd.read_csv("/content/sentiment_test.csv")
test

,title,sentiment
0,What is melatonin -- and should you take it to...,Neutral
1,The world in 2200,Neutral
2,The real story behind Archimedes' Eureka!,Neutral
3,Are you technically fit to parent?,Neutral
4,Big data is better data,Positive
...,...,...
1261,"What ""Orwellian"" really means",Neutral
1262,"1,000 TED Talks in six words",Neutral
1263,We actually have a shot at stopping the climat...,Positive
1264,The billion-dollar campaign to electrify trans...,Positive


In [ ]:
test['predicted_sentiment'] = test['title'].apply(lambda text: pipe(text)[0]['label'])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
test

,title,sentiment,predicted_sentiment
0,What is melatonin -- and should you take it to...,Neutral,NEG
1,The world in 2200,Neutral,NEG
2,The real story behind Archimedes' Eureka!,Neutral,NEG
3,Are you technically fit to parent?,Neutral,NEG
4,Big data is better data,Positive,POS
...,...,...,...
1261,"What ""Orwellian"" really means",Neutral,NEG
1262,"1,000 TED Talks in six words",Neutral,NEG
1263,We actually have a shot at stopping the climat...,Positive,POS
1264,The billion-dollar campaign to electrify trans...,Positive,POS


In [ ]:
label_mapping = {'Neutral': 'NEU', 'Negative': 'NEG', 'Positive': 'POS'}
test['sentiment_formatted'] = test['sentiment'].map(label_mapping)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score, classification_report

# Zakładamy, że dane są w pandas DataFrame, np. df
y_true = test["sentiment_formatted"]  # Poprawne etykiety
y_pred = test["predicted_sentiment"]  # Przewidywania modelu

# Obliczanie metryk
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average="weighted")  # Dostosuj 'average' do przypadku (binary/multiclass)
f1 = f1_score(y_true, y_pred, average="weighted")
classification_rep = classification_report(y_true, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Classification Report:")
print(classification_rep)

# Obliczanie ROC-AUC (tylko dla klasyfikacji binarnej lub wieloetykietowej z wartościami prawdopodobieństwa)
# Załóżmy, że `classifier` zwraca również prawdopodobieństwa w `probs`.
# probs = classifier(..., return_probas=True)  # Prawdopodobieństwa dla każdej klasy
# y_prob = [probs[i][positive_class_idx] for i in range(len(probs))]  # Wyodrębnienie prawdopodobieństw dla pozytywnej klasy
# roc_auc = roc_auc_score(y_true, y_prob, multi_class="ovr")  # Ustaw multi_class w przypadku wieloklasowego problemu
# print(f"ROC-AUC: {roc_auc}")


Accuracy: 0.5434439178515008
Recall: 0.5434439178515008
F1 Score: 0.5434439178515008
Classification Report:
              precision    recall  f1-score   support

         NEG       0.00      0.00      0.00       147
         NEU       0.00      0.00      0.00       429
         POS       1.00      1.00      1.00       690

    accuracy                           0.54      1266
   macro avg       0.33      0.33      0.33      1266
weighted avg       0.54      0.54      0.54      1266



In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("CzarnyBaranie/finetuning-sentiment-model-tedx")
model = AutoModelForSequenceClassification.from_pretrained("CzarnyBaranie/finetuning-sentiment-model-tedx")


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [ ]:
text = "Are you technically fit to parent?"  # Replace with your desired text
inputs = tokenizer(text)
tokens = tokenizer.convert_ids_to_tokens(inputs.input_ids) # Get tokens from input_ids

In [ ]:
print(inputs)
print(tokens)

{'input_ids': [0, 533, 14, 10627, 1470, 9, 2365, 1911, 21, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['<s>', 'Are', 'you', 'technically', 'fit', 'to', 'par@@', 'ent@@', '?', '</s>']


In [ ]:
# Check the type of inputs
print(type(inputs))

# For lists, use len to find the length
print(f"Length of input_ids: {len(inputs.input_ids)}")

<class 'transformers.tokenization_utils_base.BatchEncoding'>
Length of input_ids: 10


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("CzarnyBaranie/finetuning-sentiment-model-tedx")
model = AutoModelForSequenceClassification.from_pretrained("CzarnyBaranie/finetuning-sentiment-model-tedx")

batch_texts = ["This is an example sentence.", "Another sentence for testing."]
inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True)

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

predicted_class_ids = logits.argmax(dim=1).tolist()  # Get predicted class IDs
print(f"Predicted class IDs: {predicted_class_ids}")

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


Predicted class IDs: [0, 0]


In [ ]:
predicted_class_ids

[0, 0]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Wczytaj tokenizer i model
tokenizer = AutoTokenizer.from_pretrained("CzarnyBaranie/finetuning-sentiment-model-tedx")
model = AutoModelForSequenceClassification.from_pretrained("CzarnyBaranie/finetuning-sentiment-model-tedx")

# Przykładowy tekst do klasyfikacji
text = "Are you technically fit to parent?"

# Tokenizacja tekstu
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

# Przetwarzanie przez model
with torch.no_grad():
    outputs = model(**inputs)

# Wyniki - logits (surowe wyniki modelu)
logits = outputs.logits

# Konwersja logits na prawdopodobieństwa (opcjonalnie, np. softmax)
probabilities = torch.nn.functional.softmax(logits, dim=-1)

# Identyfikacja przewidywanej klasy
predicted_class = torch.argmax(probabilities, dim=-1).item()

print(f"Prawdopodobieństwa: {probabilities}")
print(f"Przewidywana klasa: {predicted_class}")


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


Prawdopodobieństwa: tensor([[9.9979e-01, 1.5580e-04, 5.5180e-05]])
Przewidywana klasa: 0


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# def classify_text(text):
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     logits = outputs.logits
#     probabilities = torch.nn.functional.softmax(logits, dim=-1)
#     predicted_class = torch.argmax(probabilities, dim=-1).item()
#     return predicted_class

def classify_text(text):
    # Tokenizacja
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Przenieś wszystkie tensory wejściowe na GPU
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Przetwarzanie przez model
    with torch.no_grad():
        outputs = model(**inputs)

    # Pobierz wyniki
    logits = outputs.logits

    # Konwersja logits na prawdopodobieństwa
    probabilities = torch.nn.functional.softmax(logits, dim=-1)

    # Znajdź klasę o najwyższym prawdopodobieństwie
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    return predicted_class


# Klasyfikacja dla całej kolumny
test['predicted_class'] = test['title'].apply(classify_text)

In [ ]:
test

,title,sentiment,predicted_sentiment,sentiment_formatted,predicted_class
0,What is melatonin -- and should you take it to...,Neutral,NEG,NEU,0
1,The world in 2200,Neutral,NEG,NEU,0
2,The real story behind Archimedes' Eureka!,Neutral,NEG,NEU,0
3,Are you technically fit to parent?,Neutral,NEG,NEU,0
4,Big data is better data,Positive,POS,POS,2
...,...,...,...,...,...
1261,"What ""Orwellian"" really means",Neutral,NEG,NEU,0
1262,"1,000 TED Talks in six words",Neutral,NEG,NEU,0
1263,We actually have a shot at stopping the climat...,Positive,POS,POS,2
1264,The billion-dollar campaign to electrify trans...,Positive,POS,POS,2


In [ ]:
label_mapping = {'Neutral': '0', 'Negative': '1', 'Positive': '2'}
test['sentiment_numeric'] = test['sentiment'].map(label_mapping)

In [ ]:
test['sentiment_numeric'] = pd.to_numeric(test['sentiment_numeric'])

In [ ]:
test

,title,sentiment,predicted_sentiment,sentiment_formatted,predicted_class,sentiment_numeric
0,What is melatonin -- and should you take it to...,Neutral,NEG,NEU,0,0
1,The world in 2200,Neutral,NEG,NEU,0,0
2,The real story behind Archimedes' Eureka!,Neutral,NEG,NEU,0,0
3,Are you technically fit to parent?,Neutral,NEG,NEU,0,0
4,Big data is better data,Positive,POS,POS,2,2
...,...,...,...,...,...,...
1261,"What ""Orwellian"" really means",Neutral,NEG,NEU,0,0
1262,"1,000 TED Talks in six words",Neutral,NEG,NEU,0,0
1263,We actually have a shot at stopping the climat...,Positive,POS,POS,2,2
1264,The billion-dollar campaign to electrify trans...,Positive,POS,POS,2,2


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score, classification_report

# Zakładamy, że dane są w pandas DataFrame, np. df
y_true = test["sentiment_numeric"]  # Poprawne etykiety
y_pred = test["predicted_class"]  # Przewidywania modelu

# Obliczanie metryk
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average="weighted")  # Dostosuj 'average' do przypadku (binary/multiclass)
f1 = f1_score(y_true, y_pred, average="weighted")
classification_rep = classification_report(y_true, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Classification Report:")
print(classification_rep)

# Obliczanie ROC-AUC (tylko dla klasyfikacji binarnej lub wieloetykietowej z wartościami prawdopodobieństwa)
# Załóżmy, że `classifier` zwraca również prawdopodobieństwa w `probs`.
# probs = classifier(..., return_probas=True)  # Prawdopodobieństwa dla każdej klasy
# y_prob = [probs[i][positive_class_idx] for i in range(len(probs))]  # Wyodrębnienie prawdopodobieństw dla pozytywnej klasy
# roc_auc = roc_auc_score(y_true, y_prob, multi_class="ovr")  # Ustaw multi_class w przypadku wieloklasowego problemu
# print(f"ROC-AUC: {roc_auc}")


Accuracy: 0.9968404423380727
Recall: 0.9968404423380727
F1 Score: 0.9968404737218712
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       429
           1       1.00      0.99      1.00       147
           2       1.00      1.00      1.00       690

    accuracy                           1.00      1266
   macro avg       1.00      1.00      1.00      1266
weighted avg       1.00      1.00      1.00      1266

